In [0]:
import numpy as np
import pandas as pd
import nltk
import spacy
from tqdm import tqdm
import os
import seaborn as sns
import matplotlib.pyplot as plt
import re, string, unicodedata
import contractions
# import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from textblob import TextBlob


In [0]:
# !pip install contractions

In [0]:
import sqlite3
import pandas as pd
connection = sqlite3.connect('yelpHotelData.db')
x1 = connection.execute("select * FROM review")

In [0]:
df = pd.read_sql("select * FROM review" ,connection)


### Adding the extra Attributes to get additional information on review length and polarity

In [0]:
#Add sentiment analysis columns
def reviewFeatures(row):
  review = row['reviewContent']
  analysis = TextBlob(review)
  return pd.Series([len(review), analysis.sentiment.polarity, analysis.sentiment.subjectivity]
           , index = ['reviewLength', 'sentimentPolarity', 'sentimentSubjectivity'])

df[['reviewLength', 'sentimentPolarity', 'sentimentSubjectivity']]  = df.apply(reviewFeatures, axis=1)


#Convert NR->N , YR->Y
df.loc[df['flagged'] == "NR", 'flagged'] = "N"
df.loc[df['flagged'] == "YR", 'flagged'] = "Y"
df.loc[df['flagged'] == "N", 'Target'] = "0"
df.loc[df['flagged'] == "Y", 'Target'] = "1"

In [0]:
df.head(10)

,date,reviewID,reviewerID,reviewContent,rating,usefulCount,coolCount,funnyCount,flagged,hotelID,reviewLength,sentimentPolarity,sentimentSubjectivity
0,6/8/2011,MyNjnxzZVTPq,IFTr6_6NI4CgCVavIL9k5g,Let me begin by saying that there are two kind...,5,18,11,28,N,tQfLGoolUMu2J0igcWcoZg,2249.0,0.025111,0.569762
1,8/30/2011,BdD7fsPqHQL73hwENEDT-Q,c_-hF15XgNhlyy_TqzmdaA,The only place inside the Loop that you can st...,3,0,3,4,N,tQfLGoolUMu2J0igcWcoZg,261.0,-0.046875,0.625000
2,6/26/2009,BfhqiyfC,CiwZ6S5ZizAFL5gypf8tLA,I have walked by the Tokyo Hotel countless tim...,5,12,14,23,N,tQfLGoolUMu2J0igcWcoZg,1239.0,0.172028,0.615385
3,9/16/2010,Ol,nf3q2h-kSQoZK2jBY92FOg,"If you are considering staying here, watch thi...",1,8,2,6,N,tQfLGoolUMu2J0igcWcoZg,80.0,0.250000,0.333333
4,2/5/2010,i4HIAcNTjabdpG1K4F5Q2g,Sb3DJGdZ4Rq__CqxPbae-g,"This place is disgusting, absolutely horrible,...",3,11,4,9,N,tQfLGoolUMu2J0igcWcoZg,251.0,-0.435000,0.505000
5,8/9/2010,veKKNAaSKWj8os,nR7zLyFOlzAYqmzgJ3DtXg,Disgusting!!! There is literally duct tape ho...,1,1,0,3,N,tQfLGoolUMu2J0igcWcoZg,452.0,-0.265420,0.556944
6,8/11/2012,6c-ZiQkHXtp1n6VfiKDQ3g,747lP4p8dUD6RTkcsIaSGg,This hotel came up on Hotwire for $108 a night...,4,2,0,1,N,33Xc1Bk_gkSY5xb2doQ7Ng,1486.0,0.111279,0.553654
7,7/9/2012,POWQ6FuUf3oe2ZkhmHvciA,Ij5t6VdwtasSkrpp9uAbKg,"Good location, really run down. I am surprised...",2,0,0,0,N,33Xc1Bk_gkSY5xb2doQ7Ng,91.0,0.211111,0.497222
8,6/19/2012,QBynYcLgIgtAd-YfnrrAtA,hSERzClUe57bCw3nCp4plA,Beautiful lobby. The rest is a dump. The eleva...,1,0,1,1,N,33Xc1Bk_gkSY5xb2doQ7Ng,602.0,0.191667,0.766667
9,9/14/2012,ELY3TK,OMm2VcGks3QL0p0n3_kPFw,Stayed here when I went to Chicago for a weddi...,3,2,0,0,N,33Xc1Bk_gkSY5xb2doQ7Ng,731.0,0.132176,0.523611


In [0]:
df_y=df['Target']

In [0]:
df = df.drop(['Target'],axis=1)

In [0]:
df_y.head()

0    0
1    0
2    0
3    0
4    0
Name: Target, dtype: object

### Splitting the Train and Test Data into 80:20 ratio

In [0]:
# Splitting the dataset into training and test data
from sklearn.model_selection import train_test_split
XTrain, XTest, yTrain, yTest = train_test_split(df, df_y, test_size = 0.2, random_state = 42)


In [0]:
print(XTrain.shape)
print(XTest.shape)
print(yTrain.shape)
print(yTest.shape)

(550663, 13)
(137666, 13)
(550663,)
(137666,)


In [0]:
# Pickle the database with new added columns
import pickle
pickle.dump( XTrain, open( "save_modifieddatabase_trainData.pkl", "wb" ) )

pickle.dump( XTest, open( "save_modifieddatabase_testData.pkl", "wb" ) )

pickle.dump( yTrain, open( "save_preprocess_trainData_labels.pkl", "wb" ) )

pickle.dump( yTest, open( "save_preprocess_testData_labels.pkl", "wb" ) )

In [0]:
import pickle

In [0]:
XTrain = pickle.load(open("save_modifieddatabase_trainData.pkl", "rb"))
XTest = pickle.load(open("save_modifieddatabase_testData.pkl", "rb"))

In [0]:
clean_1 = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
clean_2 = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
# function to clean text data
def clean_desc(desc):
    desc = [clean_1.sub("", line.lower()) for line in desc]
    desc = [clean_2.sub(" ", line) for line in desc]
    return desc


In [0]:
XTrain['cleanedToken'] = clean_desc(XTrain['reviewContent'])
XTest['cleanedToken'] = clean_desc(XTest['reviewContent'])

In [0]:
# # remove punctuation marks
# punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~.,'

# XTrain['cleanedToken'] = XTrain['reviewContent'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
# XTest['cleanedToken'] = XTest['reviewContent'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

In [0]:
# # data cleaning 
# # remove URL's from train and test
# XTrain['cleanedToken']= XTrain['cleanedToken'].apply(lambda x: re.sub(r'http\S+', '', x))
# XTest['cleanedToken']= XTest['cleanedToken'].apply(lambda x: re.sub(r'http\S+', '', x))



In [0]:
# # # convert text to lowercase
# XTrain['cleanedToken']= XTrain['cleanedToken'].str.lower()
# XTest['cleanedToken']= XTest['cleanedToken'].str.lower()

In [0]:
XTrain['cleanedToken'].head(10)

127551    Location 5 stars Staff 4 stars Food 2 stars Do...
681184    The Beauty Bar gets 4 stars for decor bar serv...
296879    Went here last night with some friends and saw...
125992    Don't buy the figure skater and mini shift dre...
272782    a good place  and art decio is the reason i ha...
338168    Up and coming for yearsand now Bartalotta's is...
35830     Dr JulianWang is great  She has been my obgyn ...
213540    Map Room is still very cool by me I just used ...
647330    I noticed a lot of people were not fans of the...
236803               Like chipotle more options longer line
Name: cleanedToken, dtype: object

In [0]:
# Loading spacy for text analysis and cleaning
nlp = spacy.load('en_core_web_sm', disable=["tagger", "parser", "ner", "textcat"])

In [0]:
# tokenization using spaCy
def tokenization(x):
    desc_tokens = []
    for i in tqdm(x):
        i = nlp(i)
        temp = []
        for j in i:
            temp.append(j.text)
        desc_tokens.append(temp)
    
    return desc_tokens
  
# function to remove stopwords
def remove_stopwords(desc):
    s = []
    for r in tqdm(desc):
        s_2 = []
        for token in r:
            if nlp.vocab[token].is_stop == True:
                continue
            else:
                s_2.append(token)
        s.append(" ".join(s_2))    
        
    return s

In [0]:
# Tokenization
XTrain['cleanedToken'] = tokenization(XTrain['cleanedToken'])
XTest['cleanedToken'] = tokenization(XTest['cleanedToken'])

100%|██████████| 137666/137666 [01:59<00:00, 1155.40it/s]


In [0]:
# Removing Stopwords
XTrain['cleanedToken'] = remove_stopwords(XTrain['cleanedToken'])
XTest['cleanedToken'] = remove_stopwords(XTest['cleanedToken'])

100%|██████████| 137666/137666 [00:17<00:00, 7842.52it/s]


In [0]:
pickle.dump( XTrain, open( "save_preprocess_trainData.pkl", "wb" ) )

pickle.dump( XTest, open( "save_preprocess_testData.pkl", "wb" ) )

In [0]:
# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
             s = [token.lemma_ for token in nlp(i)]
             output.append(' '.join(s))
    return output

In [0]:
XTrain['cleanedToken'] = lemmatization(XTrain['cleanedToken'])
XTest['cleanedToken'] = lemmatization(XTest['cleanedToken'])

In [0]:
# nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [0]:
# from nltk.stem import WordNetLemmatizer 
  
# lemmatizer = WordNetLemmatizer() 
  
# XTrain['cleanedToken'] = lemmatizer.lemmatize(XTrain['cleanedToken'])
# XTest['cleanedToken'] = lemmatizer.lemmatize(XTest['cleanedToken'])

In [0]:
import pickle
pickle.dump( XTrain, open( "save_preprocessLemmatize_trainData.pkl", "wb" ) )

pickle.dump( XTest, open( "save_preprocessLemmatize_testData.pkl", "wb" ) )

In [0]:
# XTrain['cleanedToken'] = tokenization(XTrain['cleanedToken'])
# XTest['cleanedToken'] = tokenization(XTest['cleanedToken'])

In [0]:
# pickle.dump( XTrain, open( "save_preprocess_trainData.pkl", "wb" ) )

# pickle.dump( XTest, open( "save_preprocess_testData.pkl", "wb" ) )

# pickle.dump( yTrain, open( "save_preprocess_trainData_labels.pkl", "wb" ) )

# pickle.dump( yTest, open( "save_preprocess_testData_labels.pkl", "wb" ) )




In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [0]:
vectorizer = TfidfVectorizer()
X_train_idf = vectorizer.fit_transform(XTrain['cleanedToken'])

In [0]:
X_test_idf = vectorizer.transform(XTest['cleanedToken'])

In [0]:
pickle.dump( X_train_idf, open( "save_preprocess_trainDataCSRTFIDF.pkl", "wb" ) )
pickle.dump( X_test_idf, open( "save_preprocess_testDataCSRTFIDF.pkl", "wb" ) )

In [0]:
print(X_train_idf.shape)
print(X_test_idf.shape)


(550663, 328946)
(137666, 328946)


#### Preprocessing Done

In [0]:
yTrain = pickle.load(open("save_preprocess_trainData_labels.pkl", "rb"))
yTest = pickle.load(open("save_preprocess_testData_labels.pkl", "rb"))

In [0]:
#Train Naive Bayes - On CSR matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nbModel = MultinomialNB().fit(X_train_idf, yTrain)

df = pickle.dump(nbModel, open("nbModel.pkl", 'wb'))

predicted = nbModel.predict(X_test_idf)


print(metrics.classification_report(yTest, predicted))

              precision    recall  f1-score   support

           0       0.67      0.90      0.77     83957
           1       0.66      0.30      0.41     53709

    accuracy                           0.67    137666
   macro avg       0.66      0.60      0.59    137666
weighted avg       0.66      0.67      0.63    137666



In [0]:
#Train using XGBoost on csr matrix
from xgboost import XGBClassifier

# fit model no training data
modelXGB = XGBClassifier(verbosity=3, random_state=42)
modelXGB.fit(X_train_idf, yTrain)

# make predictions for test data
y_pred = modelXGB.predict(X_test_idf)

from sklearn import metrics

print(metrics.classification_report(yTest, y_pred))

[17:54:35] DEBUG: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/gbm/gbtree.cc:146: Using tree method: 2
[17:54:50] INFO: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/tree/updater_prune.cc:89: tree pruning end, 122 extra nodes, 0 pruned nodes, max_depth=6
[17:55:01] INFO: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/tree/updater_prune.cc:89: tree pruning end, 116 extra nodes, 0 pruned nodes, max_depth=6
[17:55:11] INFO: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/tree/updater_prune.cc:89: tree pruning end, 118 extra nodes, 0 pruned nodes, max_depth=6
[17:55:22] INFO: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/tree/updater_prune.cc:89: tree pruning end, 120 extra nodes, 0 pruned nodes, max_depth=6
[17:55:33] INFO: /private/va

[18:03:35] INFO: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/tree/updater_prune.cc:89: tree pruning end, 88 extra nodes, 0 pruned nodes, max_depth=6
[18:03:46] INFO: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/tree/updater_prune.cc:89: tree pruning end, 64 extra nodes, 0 pruned nodes, max_depth=6
[18:03:58] INFO: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/tree/updater_prune.cc:89: tree pruning end, 106 extra nodes, 0 pruned nodes, max_depth=6
[18:04:10] INFO: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/tree/updater_prune.cc:89: tree pruning end, 66 extra nodes, 0 pruned nodes, max_depth=6
[18:04:21] INFO: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/tree/updater_prune.cc:89: tree pruning end, 94 extra nodes, 0 pruned nod

[18:12:14] INFO: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/tree/updater_prune.cc:89: tree pruning end, 76 extra nodes, 0 pruned nodes, max_depth=6
[18:12:25] INFO: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/tree/updater_prune.cc:89: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=6
[18:12:35] INFO: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/tree/updater_prune.cc:89: tree pruning end, 58 extra nodes, 0 pruned nodes, max_depth=6
[18:12:46] INFO: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/tree/updater_prune.cc:89: tree pruning end, 72 extra nodes, 0 pruned nodes, max_depth=6
[18:12:56] INFO: /private/var/folders/sy/ghg1rmxn5bsgz0t7bct4d7jc0000gp/T/pip-install-edy2otjc/xgboost/xgboost/src/tree/updater_prune.cc:89: tree pruning end, 70 extra nodes, 0 pruned node

In [0]:
pickle.dump(modelXGB, open("modelXGB.pkl", 'wb'))